In [1]:
import pandas as pd
import os
import wiggum as wg
import numpy as np
import itertools

In [2]:
labeled_df = wg.LabeledDataFrame('../data/iristest')

In [3]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
sepal length,float64,continuous,"[independent, dependent]",False,NaN
sepal width,float64,continuous,[dependent],False,NaN
petal length,float64,continuous,[independent],False,NaN
petal width,float64,continuous,[independent],False,NaN
class,object,categorical,[splitby],False,NaN


In [4]:
corrobj = wg.All_Pearson()
corrobj.get_trend_vars(labeled_df)
corrobj.regression_vars

[('sepal length', 'sepal width'),
 ('petal length', 'sepal length'),
 ('petal length', 'sepal width'),
 ('petal width', 'sepal length'),
 ('petal width', 'sepal width')]

In [13]:
labeled_df.get_subgroup_trends_1lev([corrobj])
labeled_df.result_df

,dependent,group_feat,independent,subgroup,subgroup_trend,subgroup_trend_strength,trend_type,agg_trend,agg_trend_strength,comparison_type
0,sepal width,class,sepal length,Iris-setosa,0.746780,0.746780,pearson_corr,-0.109369,0.109369,aggregate-subgroup
1,sepal width,class,sepal length,Iris-versicolor,0.525911,0.525911,pearson_corr,-0.109369,0.109369,aggregate-subgroup
2,sepal width,class,sepal length,Iris-virginica,0.457228,0.457228,pearson_corr,-0.109369,0.109369,aggregate-subgroup
3,sepal length,class,petal length,Iris-setosa,0.263874,0.263874,pearson_corr,0.871754,0.871754,aggregate-subgroup
4,sepal length,class,petal length,Iris-versicolor,0.754049,0.754049,pearson_corr,0.871754,0.871754,aggregate-subgroup
5,sepal length,class,petal length,Iris-virginica,0.864225,0.864225,pearson_corr,0.871754,0.871754,aggregate-subgroup
6,sepal width,class,petal length,Iris-setosa,0.176695,0.176695,pearson_corr,-0.420516,0.420516,aggregate-subgroup
7,sepal width,class,petal length,Iris-versicolor,0.560522,0.560522,pearson_corr,-0.420516,0.420516,aggregate-subgroup
8,sepal width,class,petal length,Iris-virginica,0.401045,0.401045,pearson_corr,-0.420516,0.420516,aggregate-subgroup
9,sepal length,class,petal width,Iris-setosa,0.279092,0.279092,pearson_corr,0.817954,0.817954,aggregate-subgroup


In [14]:
linreg_obj = wg.All_Linear_Trend()
labeled_df.get_subgroup_trends_1lev([corrobj,linreg_obj], replace=True)

,dependent,group_feat,independent,subgroup,subgroup_trend,subgroup_trend_strength,trend_type,agg_trend,agg_trend_strength,comparison_type
0,sepal width,class,sepal length,Iris-setosa,0.746780,0.746780,pearson_corr,-0.109369,0.109369,aggregate-subgroup
1,sepal width,class,sepal length,Iris-setosa,0.746780,0.746780,pearson_corr,-0.109369,0.109369,aggregate-subgroup
2,sepal width,class,sepal length,Iris-versicolor,0.525911,0.525911,pearson_corr,-0.109369,0.109369,aggregate-subgroup
3,sepal width,class,sepal length,Iris-versicolor,0.525911,0.525911,pearson_corr,-0.109369,0.109369,aggregate-subgroup
4,sepal width,class,sepal length,Iris-virginica,0.457228,0.457228,pearson_corr,-0.109369,0.109369,aggregate-subgroup
5,sepal width,class,sepal length,Iris-virginica,0.457228,0.457228,pearson_corr,-0.109369,0.109369,aggregate-subgroup
6,sepal width,class,sepal length,Iris-setosa,0.746780,0.746780,pearson_corr,-0.109369,0.109369,aggregate-subgroup
7,sepal width,class,sepal length,Iris-setosa,0.746780,0.746780,pearson_corr,-0.109369,0.109369,aggregate-subgroup
8,sepal width,class,sepal length,Iris-versicolor,0.525911,0.525911,pearson_corr,-0.109369,0.109369,aggregate-subgroup
9,sepal width,class,sepal length,Iris-versicolor,0.525911,0.525911,pearson_corr,-0.109369,0.109369,aggregate-subgroup


# Test diff setttings

In [15]:
labeled_df2 = labeled_df

In [16]:
corr_vars = ['petal length', 'sepal length', 'petal width', 'sepal width']
labeled_df2.set_roles({v:['independent','dependent'] for v in corr_vars})

/home/smb/anaconda3/lib/python3.6/site-packages/wiggum/labeled_dataframe.py:365: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['role'][k] = v


In [17]:
labeled_df2.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
sepal length,float64,continuous,"[independent, dependent]",False,NaN
sepal width,float64,continuous,"[independent, dependent]",False,NaN
petal length,float64,continuous,"[independent, dependent]",False,NaN
petal width,float64,continuous,"[independent, dependent]",False,NaN
class,object,categorical,[splitby],False,NaN


In [20]:
linreg_obj2 = wg.All_Linear_Trend()
labeled_df2.get_subgroup_trends_1lev([linreg_obj2], replace=True)

/home/smb/anaconda3/lib/python3.6/site-packages/wiggum/trend_components/regression.py:134: UserWarning: cannot compute
  warnings.warn('cannot compute')


,dependent,group_feat,independent,subgroup,subgroup_trend,subgroup_trend_strength,trend_type,agg_trend,agg_trend_strength,comparison_type
0,sepal length,class,petal length,Iris-setosa,0.263874,0.263874,pearson_corr,0.871754,0.871754,aggregate-subgroup
1,sepal length,class,petal length,Iris-setosa,0.263874,0.263874,pearson_corr,0.871754,0.871754,aggregate-subgroup
2,sepal length,class,petal length,Iris-setosa,0.263874,0.263874,pearson_corr,0.871754,0.871754,aggregate-subgroup
3,sepal length,class,petal length,Iris-versicolor,0.754049,0.754049,pearson_corr,0.871754,0.871754,aggregate-subgroup
4,sepal length,class,petal length,Iris-versicolor,0.754049,0.754049,pearson_corr,0.871754,0.871754,aggregate-subgroup
5,sepal length,class,petal length,Iris-versicolor,0.754049,0.754049,pearson_corr,0.871754,0.871754,aggregate-subgroup
6,sepal length,class,petal length,Iris-virginica,0.864225,0.864225,pearson_corr,0.871754,0.871754,aggregate-subgroup
7,sepal length,class,petal length,Iris-virginica,0.864225,0.864225,pearson_corr,0.871754,0.871754,aggregate-subgroup
8,sepal length,class,petal length,Iris-virginica,0.864225,0.864225,pearson_corr,0.871754,0.871754,aggregate-subgroup
9,sepal length,class,petal length,Iris-setosa,0.263874,0.263874,pearson_corr,0.871754,0.871754,aggregate-subgroup


In [21]:
linreg_obj2.regression_vars

[('sepal length', 'sepal width'),
 ('sepal length', 'petal length'),
 ('sepal length', 'petal width'),
 ('sepal width', 'sepal length'),
 ('sepal width', 'petal length'),
 ('sepal width', 'petal width'),
 ('petal length', 'sepal length'),
 ('petal length', 'sepal width'),
 ('petal length', 'petal width'),
 ('petal width', 'sepal length'),
 ('petal width', 'sepal width'),
 ('petal width', 'petal length')]

# Test object

In [5]:
indep, dep = zip(*corrobj.regression_vars)
corr_var_list = list(set(indep))
corr_var_list.extend(list(set(dep)))

corr_var_list = list(set(corr_var_list))
corr_var_list

['petal length', 'sepal length', 'petal width', 'sepal width']

In [6]:
corr_mat = labeled_df.df[corr_var_list].corr()

In [7]:
corr_mat

,petal length,sepal length,petal width,sepal width
petal length,1.000000,0.871754,0.962757,-0.420516
sepal length,0.871754,1.000000,0.817954,-0.109369
petal width,0.962757,0.817954,1.000000,-0.356544
sepal width,-0.420516,-0.109369,-0.356544,1.000000


In [8]:
gby_df = labeled_df.df.groupby('class')
corr_mat_gby = gby_df.corr()
groupby_vars = list(gby_df.groups.keys())

sg_corr_target_vals = [(i,d, corr_mat_gby[i][g][d],g) for (i,d),g in
                                        itertools.product(corrobj.regression_vars,groupby_vars)]
sg_corr_target_vals

[('sepal length', 'sepal width', 0.7467803732639269, 'Iris-setosa'),
 ('sepal length', 'sepal width', 0.5259107172828243, 'Iris-versicolor'),
 ('sepal length', 'sepal width', 0.45722781639411286, 'Iris-virginica'),
 ('petal length', 'sepal length', 0.26387409291868696, 'Iris-setosa'),
 ('petal length', 'sepal length', 0.7540489585920161, 'Iris-versicolor'),
 ('petal length', 'sepal length', 0.8642247329355762, 'Iris-virginica'),
 ('petal length', 'sepal width', 0.17669462869680694, 'Iris-setosa'),
 ('petal length', 'sepal width', 0.5605220916929818, 'Iris-versicolor'),
 ('petal length', 'sepal width', 0.4010445773427851, 'Iris-virginica'),
 ('petal width', 'sepal length', 0.27909157499959686, 'Iris-setosa'),
 ('petal width', 'sepal length', 0.5464610715986299, 'Iris-versicolor'),
 ('petal width', 'sepal length', 0.28110770915731953, 'Iris-virginica'),
 ('petal width', 'sepal width', 0.27997288851690444, 'Iris-setosa'),
 ('petal width', 'sepal width', 0.6639987200241115, 'Iris-versicolo

In [9]:
[(i,d,g) for (i,d),g in
                                        itertools.product(corrobj.regression_vars,groupby_vars)]

[('sepal length', 'sepal width', 'Iris-setosa'),
 ('sepal length', 'sepal width', 'Iris-versicolor'),
 ('sepal length', 'sepal width', 'Iris-virginica'),
 ('petal length', 'sepal length', 'Iris-setosa'),
 ('petal length', 'sepal length', 'Iris-versicolor'),
 ('petal length', 'sepal length', 'Iris-virginica'),
 ('petal length', 'sepal width', 'Iris-setosa'),
 ('petal length', 'sepal width', 'Iris-versicolor'),
 ('petal length', 'sepal width', 'Iris-virginica'),
 ('petal width', 'sepal length', 'Iris-setosa'),
 ('petal width', 'sepal length', 'Iris-versicolor'),
 ('petal width', 'sepal length', 'Iris-virginica'),
 ('petal width', 'sepal width', 'Iris-setosa'),
 ('petal width', 'sepal width', 'Iris-versicolor'),
 ('petal width', 'sepal width', 'Iris-virginica')]

In [10]:
sg = corrobj.get_trends(gby_df,'subgroup_trend')
sg

,independent,dependent,subgroup_trend,subgroup_trend_strength,subgroup,group_feat,trend_type
0,sepal length,sepal width,0.746780,0.746780,Iris-setosa,class,pearson_corr
1,sepal length,sepal width,0.525911,0.525911,Iris-versicolor,class,pearson_corr
2,sepal length,sepal width,0.457228,0.457228,Iris-virginica,class,pearson_corr
3,petal length,sepal length,0.263874,0.263874,Iris-setosa,class,pearson_corr
4,petal length,sepal length,0.754049,0.754049,Iris-versicolor,class,pearson_corr
5,petal length,sepal length,0.864225,0.864225,Iris-virginica,class,pearson_corr
6,petal length,sepal width,0.176695,0.176695,Iris-setosa,class,pearson_corr
7,petal length,sepal width,0.560522,0.560522,Iris-versicolor,class,pearson_corr
8,petal length,sepal width,0.401045,0.401045,Iris-virginica,class,pearson_corr
9,petal width,sepal length,0.279092,0.279092,Iris-setosa,class,pearson_corr


In [11]:
df = corrobj.get_trends(labeled_df.df,'agg_trend')
df

,independent,dependent,agg_trend,agg_trend_strength,trend_type
0,sepal length,sepal width,-0.109369,0.109369,pearson_corr
1,petal length,sepal length,0.871754,0.871754,pearson_corr
2,petal length,sepal width,-0.420516,0.420516,pearson_corr
3,petal width,sepal length,0.817954,0.817954,pearson_corr
4,petal width,sepal width,-0.356544,0.356544,pearson_corr


In [12]:
pd.merge(sg,df)

,independent,dependent,subgroup_trend,subgroup_trend_strength,subgroup,group_feat,trend_type,agg_trend,agg_trend_strength
0,sepal length,sepal width,0.746780,0.746780,Iris-setosa,class,pearson_corr,-0.109369,0.109369
1,sepal length,sepal width,0.525911,0.525911,Iris-versicolor,class,pearson_corr,-0.109369,0.109369
2,sepal length,sepal width,0.457228,0.457228,Iris-virginica,class,pearson_corr,-0.109369,0.109369
3,petal length,sepal length,0.263874,0.263874,Iris-setosa,class,pearson_corr,0.871754,0.871754
4,petal length,sepal length,0.754049,0.754049,Iris-versicolor,class,pearson_corr,0.871754,0.871754
5,petal length,sepal length,0.864225,0.864225,Iris-virginica,class,pearson_corr,0.871754,0.871754
6,petal length,sepal width,0.176695,0.176695,Iris-setosa,class,pearson_corr,-0.420516,0.420516
7,petal length,sepal width,0.560522,0.560522,Iris-versicolor,class,pearson_corr,-0.420516,0.420516
8,petal length,sepal width,0.401045,0.401045,Iris-virginica,class,pearson_corr,-0.420516,0.420516
9,petal width,sepal length,0.279092,0.279092,Iris-setosa,class,pearson_corr,0.817954,0.817954
